In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import (
    register_data_set,
)  # , register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import (
    RedshiftPointCasROIHeads,
    RedshiftPointROIHeads,
    RedshiftPDFROIHeads,
    return_lazy_model,
)
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser

In [2]:
# import importlib
# importlib.reload(rail.estimation.algos.deepdisc)

In [3]:
# from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle
from rail.core.stage import RailStage

from rail.deepdisc.configs import *

In [4]:
#/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train_small.npy
cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"
dirpath = "./tests/deepdisc/test_data/dc2/"
output_dir = "./"
output_name = "test"

#trainfile = dirpath + "flattened_data_test.npy"
trainfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5"
testfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test.hdf5"
metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/train_metadata.json"
test_metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata.json"

classes = ["object"]
numclasses = 1

In [5]:
cfg = get_lazy_config(cfgfile, 1, 1)
cfg.train.init_checkpoint

'detectron2://ImageNetPretrained/MSRA/R-50.pkl'

In [6]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [7]:
from deepdisc.data_format.file_io import get_data_from_json

#testdata = np.load(trainfile, allow_pickle=True)
#testdata = DS.read_file("testdata", TableHandle, trainfile)
metadata = get_data_from_json(metadatafile)
test_metadata = get_data_from_json(test_metadatafile)

In [12]:
#training = DS.add_data("training", testdata(), TableHandle, path=trainfile) #()["images"]
#testing = DS.add_data("testing", testdata, TableHandle)

training = DS.add_data("training", data=None, handle_class=TableHandle, path=trainfile) #()["images"]
testing = DS.add_data("testing", data=None, handle_class=TableHandle, path=testfile)

metadatahandle = DS.add_data("metadata", metadata, JsonHandle, path=metadatafile)
#metadatahandle = DS.add_data("test_metadata", test_metadata, JsonHandle, path=test_metadatafile)

In [9]:
deep_dict = dict(
    chunk_size=100,
    epochs=20,
    numclasses=1,
    batch_size=1,
    output_dir="./",
    cfgfile="/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py",
    output_name="test_informer",
#     hdf5_groupname="images",
    
)

In [13]:
# Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='detectron2://ImageNetPretrained/MSRA/R-50.pkl', **deep_dict)
Inform = DeepDiscInformer.make_stage(
    name="Inform_DeepDISC", model="test_informer.pkl", **deep_dict
)

In [14]:
Inform.inform(training, metadatahandle)

caching data
[12/06 21:11:37 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/06 21:11:37 d2.data.common]: Serializing 1000 elements to byte tensors and concatenating them all ...
[12/06 21:12:01 d2.data.common]: Serialized dataset takes 274.77 MiB
[12/06 21:12:01 d2.data.build]: Making batched data loader with batch_size=1
[12/06 21:12:01 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/06 21:12:01 d2.data.common]: Serializing 1000 elements to byte tensors and concatenating them all ...
[12/06 21:12:08 d2.data.common]: Serialized dataset takes 274.77 MiB
[12/06 21:12:08 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://ImageNetPretrained/MSRA/R-50.pkl ...
[12/06 21:12:08 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
WARNING [12/06 21:12:08 d2.checkpoint.c2_model_loading]: Shape of stem.conv1.weight in checkpoint is tor

Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.stem.conv1.weight
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight

Model training:
[12/06 21:12:09 d2.engine.train_loop]: Starting training from iteration 0
Iteration:  5  time:  4.260800778865814e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.6661180257797241, 0.19525235891342163, 0.10094676911830902, 0.6357252597808838, 0.18379457294940948] val loss:  0 lr:  [0.001]
Iteration:  10  time:  4.100147634744644e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.6154966950416565, 0.22101737558841705, 0.1035844087600708, 0.4806888997554779, 0.16354289650917053] val loss:  0 lr:  [0.001]
Iteration:  15  time:  7.299240678548813e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.5481612682342529, 0.25607243180274963, 0.10527437925338745, 0.2947094440460205, 0.20163220167160034] val loss:  0 lr:  [0.001]
Iteration:  20  time:  3.988388925790787e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'lo

### Inference

In [15]:
metadatahandle = DS.add_data("metadata", test_metadata, JsonHandle, path=test_metadatafile)

In [16]:
# Estimator = DeepDiscEstimator.make_stage(name='DeepDiscEstimator',
#                                       model=Inform.get_handle('model'), **deep_dict)

Estimator = DeepDiscPDFEstimator.make_stage(
    name="DeepDiscEstimator",
    model=Inform.get_handle("model"),
    #hdf5_groupname="images",
    **deep_dict,
)

In [ ]:
results = Estimator.estimate(testing, metadatahandle)

caching data
[12/06 21:13:50 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./test_informer.pth ...
Processing Data
Matching objects


In [31]:
res = results.read()

NameError: name 'results' is not defined